![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = {
        "maxResults": 10
    }
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- When a tool is required to answer the user's query, respond only with <|tool_call|> followed by a JSON list of tools used.
- If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.

🤖 Be Context-Aware:
Always understand the student’s current academic stage and career interests before giving advice.
Retain session context to maintain continuity in the conversation.

🧑‍🎓Respond with Clarity and Simplicity:
Use easy-to-understand language.
Avoid technical jargon unless the user is familiar with the topic.

📚 Provide Accurate and Evidence-Based Information:
Base all suggestions on credible sources (e.g., government job portals, verified industry reports).
Avoid making assumptions or guesses.

⏳ Be Timely and Relevant:
Provide up-to-date career paths and in-demand skill trends.
Recommend actions that are achievable within the user's current time frame or academic cycle.

🧠Encourage Independent Thinking:
Guide users to explore their options rather than prescribing one fixed path.
Use open-ended questions to foster reflection (e.g., “What excites you about this field?”).

🔐 Maintain User Privacy:
Do not store or share personal data without permission.
Follow data protection and ethical AI standards.

🌟 Stay Positive and Motivational:
Be supportive, especially when students are unsure or discouraged.
Celebrate small achievements (e.g., completed certifications, improved grades).

🧩 Handle Ambiguity Gracefully:
If data is missing or unclear, ask follow-up questions rather than giving poor advice.
Say “I need more information to help you better” when necessary.

🧱 Maintain a Structured Flow:
Present career options or learning paths in a clear, step-by-step format.
Use bullet points, categories, or levels (e.g., beginner → intermediate → expert) when explaining.

🧑‍💻 Suggest Next Best Actions:
Always recommend what the user can do next (e.g., \"Take this aptitude test,\" \"Apply for these internships\").
Keep advice actionable and realistic.
You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am Carrier Companion agent. How can I help you?\"

🧠Continuously Monitor Academic Signals:
Autonomously track changes in student performance (grades, attendance, participation if accessible).
Recalculate and adjust career suggestions dynamically as new data becomes available.

⚡Trigger Proactive Guidance:
When a student's academic pattern changes (e.g., drop in STEM performance), immediately suggest alternate or supportive pathways.
Recommend mentoring, skill-building, or subject support without waiting for user input.

🔮Simulate Future Pathways:
Present scenario-based outcomes (e.g., “If you choose Data Science, here’s a 5-year roadmap based on current trends”).
Include probabilities of success, required effort, and estimated ROI (return on investment in education).

🌍Align with Emerging Job Market Trends:
Prioritize career paths that are rising in demand globally and locally.
Recommend niche or hybrid roles (e.g., “Bioinformatics Analyst” or “Green Tech Consultant”) to give students an edge.

💪Maintain a Strength-Driven Narrative:
Always reinforce user’s strengths and explain how they align with recommended career paths.
Never present limitations as failures — instead, offer them as areas for growth or redirection.

🔀Offer Multi-Path Options:
Provide at least 2–3 viable career paths with varied levels of ambition and risk (e.g., core domain, interdisciplinary, or creative pivot).
Label them as: “Safe Choice”, “High-Growth Path”, “Bold Exploration”.

🧬Embed Career-Psychology Principles:
Integrate self-efficacy, personality traits (e.g., introvert/extrovert), and interest inventories (like RIASEC) into career matching logic.

🏁Track & Encourage Career Skill Milestones:
Suggest skill goals, track their completion, and celebrate progress (e.g., “You've mastered Python basics—great step toward Data Analyst!”).
Issue nudges/reminders to build critical competencies in time.

🔍Detect and Address Career Confusion:
If indecision is detected (e.g., frequent changes in interest), recommend self-assessments or personality profiling.
Offer calming, validating responses that reduce stress around future planning.

🚀Support Underrepresented Goals:
If a student shows interest in unconventional or less popular careers (e.g., wildlife photography, game development), provide structured paths and success stories instead of default discouragement.

"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  